In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [2]:
# Load both the ratings and movies data into a pandas DataFrame
ratings_df = pd.read_csv('./ml-25m/ratings.csv', rownum = 5000, 
                         usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
movies_df = pd.read_csv('./ml-25m/movies.csv', rownum = 5000, 
                        usecols=['movieId', 'title', 'genres'], dtype={'movieId': 'int32', 'title': 'str', 'genres': 'str'})

TypeError: read_csv() got an unexpected keyword argument 'rownum'

In [ ]:
# One-hot encode genres
genres_df = movies_df['genres'].str.get_dummies(sep='|')
movies_df = pd.concat([movies_df.drop('genres', axis=1), genres_df], axis=1)
movies_df.head()

In [ ]:
# Merge ratings on movies
merge_df = ratings_df.merge(movies_df, on='movieId')

# Define Reader for min. and max. ratings and load data using Surprise Dataset
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(merge_df[['userId', 'movieId', 'rating']], reader)

In [ ]:
train, test = train_test_split(data, test_size=0.2)

In [ ]:
# Define SVD model
model = SVD(n_factors=50, n_epochs=20, reg_all=0.02, lr_all=0.005)

In [ ]:
# Fit data and make predictions
model.fit(train)
predictions = model.test(test)

In [ ]:
# Evaluate the model using RMSE
rmse = accuracy.rmse(predictions)

print("Root Mean Squared Error = {:.4f}".format(rmse))